In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet

root_dir = "shopee-product-matching/"
target_shape = (400, 400)

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import tensorflow.keras as keras
import tensorflow as tf

from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
import io, os
import numpy as np
import pandas as pd

import tensorflow.keras as keras
import tensorflow as tf
#import tensorflow_datasets as tfds
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

In [ ]:
!pwd
!mkdir ~/.kaggle
!echo '{"username":"polololya","key":"1f4ad1fc8a1535e236393da48a325aa8"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!cat ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c shopee-product-matching
!mkdir shopee-product-matching
!unzip -qq shopee-product-matching.zip -d shopee-product-matching

In [ ]:
def preprocess_image(filename):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """

    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    return image


def preprocess_triplets(anchor, positive, negative):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(anchor),
        preprocess_image(positive),
        preprocess_image(negative),
    )

In [ ]:
class DatasetHandler:
    def __init__(self, df_path, img_dir, validation_split=0.2):
        all_train_df = pd.read_csv(df_path)
        all_train_df['image'] = img_dir + all_train_df['image']
        self.isEmpty = True
        self.df = all_train_df
       # self.train_df = pd.DataFrame()
       # self.val_df = pd.DataFrame()
        self.train_df = tf.data.Dataset
        self.val_df = tf.data.Dataset
        self.validation_split = validation_split
        
    def form_triplet(self, ind):
        anchor = self.df.iloc[ind]
        similar = self.df[self.df["label_group"] == anchor["label_group"]][self.df["posting_id"] != anchor["posting_id"]].index
        different = self.df[self.df["label_group"] != anchor["label_group"]].index
        
        positive = self.df.loc[np.random.choice(similar)]
        negative = self.df.loc[np.random.choice(different)]
            
        return(anchor, positive, negative)
        
    def generate_triplets(self):
        #triplets = {'anchors': [], 'positive': [], 'negative': []}
        anchors = []
        positive = []
        negative = []
        for i in range(len(self.df)):
            a, p, n = self.form_triplet(i)
           # triplets['anchors'].append(a["image"])
           # triplets['positive'].append(p["image"])
           # triplets['negative'].append(n["image"])
            anchors.append(a["image"])
            positive.append(p["image"])
            negative.append(n["image"])
            print(f"Generated triplet {i+1} out of {len(self.df)}")
        #triplets_df = pd.DataFrame(triplets)
        anchor_dataset = tf.data.Dataset.from_tensor_slices(anchor_images)
        positive_dataset = tf.data.Dataset.from_tensor_slices(positive_images)
        negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)
        dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))
        dataset = dataset.map(preprocess_triplets)

        train_dataset = dataset.take(round(image_count * 0.8))
        val_dataset = dataset.skip(round(image_count * 0.8))

        train_dataset = train_dataset.batch(32, drop_remainder=False)
        train_dataset = train_dataset.prefetch(8)

        val_dataset = val_dataset.batch(32, drop_remainder=False)
        val_dataset = val_dataset.prefetch(8)

        self.train_df = train_dataset
        self.val_df = val_dataset
       # self.train_df, self.val_df = train_test_split(dataset, test_size=self.validation_split)
            
    def get_training_data(self):
        if self.isEmpty:
            self.generate_triplets()
            self.isEmpty = False
        return self.train_df
    
    def get_validation_data(self):
        if self.isEmpty:
            self.generate_triplets()
            self.isEmpty = False
        return self.val_df

In [ ]:
df_path = "shopee-product-matching/train.csv"
img_dir = "shopee-product-matching/train_images/"

dh = DatasetHandler(df_path, img_dir)
train_data = dh.get_training_data()
val_data = dh.get_validation_data()

In [86]:
base_cnn = resnet.ResNet50(
    weights="imagenet", input_shape=target_shape + (3,), include_top=False
)

flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

embedding = Model(base_cnn.input, output, name="Embedding")

trainable = False
for layer in base_cnn.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

In [87]:
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
positive_input = layers.Input(name="positive", shape=target_shape + (3,))
negative_input = layers.Input(name="negative", shape=target_shape + (3,))

distances = DistanceLayer()(
    embedding(resnet.preprocess_input(anchor_input)),
    embedding(resnet.preprocess_input(positive_input)),
    embedding(resnet.preprocess_input(negative_input)),
)

siamese_network = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

In [88]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]


In [89]:
siamese_model = SiameseModel(siamese_network)
siamese_model.compile(optimizer=optimizers.Adam(0.0001))
siamese_model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10
680/680 [==============================] - 2358s 3s/step - loss: 0.3345 - val_loss: 0.2116
Epoch 2/10
680/680 [==============================] - 2297s 3s/step - loss: 0.1854 - val_loss: 0.1839
Epoch 3/10
680/680 [==============================] - 2289s 3s/step - loss: 0.1623 - val_loss: 0.1922
Epoch 4/10
680/680 [==============================] - 2284s 3s/step - loss: 0.1397 - val_loss: 0.1810
Epoch 5/10
680/680 [==============================] - 2306s 3s/step - loss: 0.1306 - val_loss: 0.1956
Epoch 6/10
 33/680 [>.............................] - ETA: 25:49 - loss: 0.1152

KeyboardInterrupt: ignored